<a href="https://colab.research.google.com/github/aadishchopra/NLP/blob/main/Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
torch.cuda.is_available()

True

In [24]:
#from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
!pip install transformers datasets huggingface_hub
!apt-get install git-lfs
import pandas as pd
import datasets
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from sklearn.model_selection import train_test_split


pd.set_option('display.max_colwidth', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [25]:
df=pd.read_csv('/content/tweets/Corona_NLP_train.csv', encoding='latin-1',)
#df=df[1:500]
df_train,df_test=train_test_split(df,test_size=0.3)
df_train.head()



,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
20468,24267,69219,United States,25-03-2020,Look at the massive amount of CEOs which are all pedophiles have stepped down,Neutral
36278,40077,85029,NaN,09-04-2020,"In Praise of Preppers: Thanks to a community that has literally been waiting for this day to come, thereÂs now an online shopping industry to help you prepare for the COVID-19 apocalypse. https://t.co/c9FsNMaC7n",Extremely Positive
15665,19464,64416,Australia,22-03-2020,best recipe for handmade sanitizer ... it really works! #Coronavirus .... https://t.co/bTaoQMBAn1,Extremely Positive
38348,42147,87099,"Boston, MA",11-04-2020,Last week the amp took unprecedented action to try and stop related scam from bombarding American consumers demanding that gateway providers cut off robocallers in 48 hours or risk losing all access to U S phone network,Extremely Negative
32507,36306,81258,Makati City,07-04-2020,The rise in consumer prices eased to 2.5 percent year-on-year in March due mainly to the price freeze put in place by the government under the nationwide state of calamity amid the COVID-19 pandemic and the decline in the global prices of oil. https://t.co/YKjhJhs9eE,Positive


In [26]:
df_train=df_train[['OriginalTweet','Sentiment']]
df_train['Sentiment'].unique()

#Creating a new column for labelling the dataset in decimal numbers
emotion=['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive']
num=[0,1,2,3,4]

emotion_dict=dict(zip(emotion,num))


emotion_dict['Positive']

1

In [27]:
df_test=df_test[['OriginalTweet','Sentiment']]


In [28]:
df_dict_train=datasets.Dataset.from_dict(df_train)
df_dict_test=datasets.Dataset.from_dict(df_test)

In [30]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased')

#from IPython.core.debugger import set_trace

def letstokenize(text):
    #set_trace()
    o_tweet=text['OriginalTweet']
    sentiment=text['Sentiment']
    encoding=tokenizer(o_tweet,padding="max_length",truncation=True,max_length=128)
    #label=text['Sentiment']
    encoding["labels"]=emotion_dict[sentiment]
    return encoding

encoded_df=df_dict_train.map(letstokenize)
encoded_df_test=df_dict_test.map(letstokenize)

Map:   0%|          | 0/28809 [00:00<?, ? examples/s]

Map:   0%|          | 0/12348 [00:00<?, ? examples/s]

In [32]:
distilbert=AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                             num_labels=len(emotion),
                                                             )

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [33]:
batch_size=8
metric_name='f1'

repo_name='distilbert-tweets-analysis1'

args=TrainingArguments(output_dir=repo_name,
                  evaluation_strategy='epoch',
                  metric_for_best_model=metric_name,
                  per_device_train_batch_size=batch_size,
                  num_train_epochs=4,
                  load_best_model_at_end=True,
                  #no_cuda=True,
                  per_device_eval_batch_size=batch_size,
                  weight_decay=0.1,
                  save_strategy='epoch',
                  learning_rate=1e-5,
                  #report_to="none" 
                  push_to_hub=True
                      )

In [34]:
!pip install evaluate
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score

'''
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy",average='micro')
    load_f1 = evaluate.load("f1",average='micro') 
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
'''

def compute_metrics (eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    return metric.compute(predictions=preds, references = labels, average="micro")

In [36]:
trainer = Trainer(
    distilbert,
    args,
    train_dataset=encoded_df,
    eval_dataset=encoded_df_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    )

Cloning https://huggingface.co/aadishhug/distilbert-tweets-analysis1 into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Download file runs/Mar15_05-31-53_6c08dea5a836/1678858361.7021003/events.out.tfevents.1678858361.6c08dea5a836.…

Download file runs/Mar15_05-31-53_6c08dea5a836/events.out.tfevents.1678860270.6c08dea5a836.186.8: 100%|#######…

...

Clean file training_args.bin:  29%|##9       | 1.00k/3.43k [00:00<?, ?B/s]

Clean file runs/Mar15_05-31-53_6c08dea5a836/1678858361.7021003/events.out.tfevents.1678858361.6c08dea5a836.186…

Clean file runs/Mar15_05-31-53_6c08dea5a836/events.out.tfevents.1678860270.6c08dea5a836.186.8: 100%|##########…

Clean file runs/Mar15_05-31-53_6c08dea5a836/events.out.tfevents.1678858361.6c08dea5a836.186.6:  10%|#         …

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [37]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.687600,0.709711,0.729106
2,0.521300,0.569970,0.791140
3,0.417700,0.523412,0.830742
4,0.321900,0.568801,0.833495


TrainOutput(global_step=14408, training_loss=0.5488591689258598, metrics={'train_runtime': 1816.651, 'train_samples_per_second': 63.433, 'train_steps_per_second': 7.931, 'total_flos': 3816457460259840.0, 'train_loss': 0.5488591689258598, 'epoch': 4.0})

In [38]:
trainer.evaluate()

{'eval_loss': 0.5688012838363647,
 'eval_f1': 0.833495302883058,
 'eval_runtime': 49.3864,
 'eval_samples_per_second': 250.028,
 'eval_steps_per_second': 31.264,
 'epoch': 4.0}

In [39]:
trainer.push_to_hub()

Upload file runs/Mar15_17-18-22_6f17d5992f6a/events.out.tfevents.1678900808.6f17d5992f6a.128.0: 100%|#########…

Upload file runs/Mar15_17-18-22_6f17d5992f6a/events.out.tfevents.1678902736.6f17d5992f6a.128.2: 100%|#########…

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/aadishhug/distilbert-tweets-analysis1
   aa27558..873b8c4  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/aadishhug/distilbert-tweets-analysis1
   aa27558..873b8c4  main -> main

To https://huggingface.co/aadishhug/distilbert-tweets-analysis1
   873b8c4..01e843c  main -> main

   873b8c4..01e843c  main -> main



'https://huggingface.co/aadishhug/distilbert-tweets-analysis1/commit/873b8c4f15246d29876f6bdfd7659ab63fad4858'

In [40]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="aadishhug/distilbert-tweets-analysis1")

sentiment_model(['TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1'])

[{'label': 'LABEL_2', 'score': 0.8468700051307678}]